# pmc sentenceのelasticsearchを作成する

## まずはデータの確認

In [1]:
import pandas as pd
import sqlite3
import glob
import tqdm

In [2]:
paths = glob.glob("/workspace/99-NAS_data/pmc_24/04_data_from_pmc_sent/04_data_from_pmc_sent/*.tsv")
print(len(paths))

321


In [3]:
df = pd.read_csv(paths[0], sep="\t")
df.head()

,pmc_id,sentence_index,section,text
0,2136584,1,abstract,Quantitative studies have demonstrated that a...
1,2136584,2,abstract,The augmentation of antibody levels varied fro...
2,2136584,3,abstract,This antibody-enhancing property was found to ...
3,2136584,4,abstract,Factors affecting this enhancement were invest...
4,2136584,5,abstract,The data obtained are discussed in relation to...


In [4]:
pmc_id = set()
for path in tqdm.tqdm(paths):
    df = pd.read_csv(path, sep="\t")
    pmc_id.update(set(df["pmc_id"].unique().tolist()))
print(len(pmc_id))

 15%|█▍        | 47/321 [02:54<23:11,  5.08s/it]/tmp/ipykernel_707198/2904170728.py:3: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep="\t")
 57%|█████▋    | 182/321 [09:27<05:45,  2.48s/it]/tmp/ipykernel_707198/2904170728.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep="\t")
 99%|█████████▉| 318/321 [17:09<00:13,  4.39s/it]/tmp/ipykernel_707198/2904170728.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep="\t")
100%|██████████| 321/321 [17:27<00:00,  3.26s/it]

2500866


In [5]:
sql_paths = glob.glob("/workspace/99-NAS_data/pmc_25/db/db/*.db")
print(len(sql_paths))

2089


In [6]:
sql_pmc_id = set()
for sql_path in tqdm.tqdm(sql_paths):
    conn = sqlite3.connect(sql_path)
    query = "SELECT DISTINCT PMCID FROM raw"
    df_sql = pd.read_sql_query(query, conn)
    sql_pmc_id.update(set(df_sql["PMCID"].unique().tolist()))
    conn.close()
print(len(sql_pmc_id))

100%|██████████| 2089/2089 [10:35<00:00,  3.29it/s]

6749276


In [7]:
df_sql.head()

,PMCID
0,PMC11574378
1,PMC11574379
2,PMC11574380
3,PMC11574381
4,PMC11574382


In [8]:
sql_pmc_id_processed = set()
for pmcid in sql_pmc_id:
    sql_pmc_id_processed.add(pmcid.replace("PMC", "").strip())

In [9]:
print(len(sql_pmc_id_processed), len(pmc_id))

6749276 2500866


In [10]:
overlap = pmc_id.intersection(sql_pmc_id_processed)
print(len(overlap))

2255


In [14]:
pmc_24_id = set()
for i in pmc_id:
    try:
        pmc_24_id.add(int(i))
    except:
        continue

pmc_25_id = set()
for i in sql_pmc_id_processed:
    try:
        pmc_25_id.add(int(i)) 
    except:
        continue

In [ ]:
print(len(pmc_24_id), len(pmc_25_id))
print(len(pmc_24_id.intersection(pmc_25_id)))

2500854 6749275
2500551


In [17]:
has_sentence = pmc_25_id - pmc_24_id
no_sentence = pmc_25_id - has_sentence
print(len(has_sentence), len(no_sentence))

4248724 2500551


In [23]:
sql_pmc_id == sorted(sql_pmc_id)

False

In [25]:
print(min(pmc_25_id))

176545
